In [5]:
import math
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np
from numpy import nan as NA
import csv
from matplotlib import pyplot as plt
f = open("EuCitiesTemperatures.csv")
data = pd.read_csv(f)
lst=Series(list(data['latitude']), index=data['country'].values)
newlatlist=[]
for i in data.index:
    if math.isnan(data['latitude'][i]):
        newlatlist.append(round(np.nanmean(np.array(lst[data['country'][i]])),2))
    else:
        newlatlist.append(data['latitude'][i])
data['latitude']=newlatlist
lst1=Series(list(data['longitude']), index=data['country'].values)
newlonglist=[]
for i in data.index:
    if math.isnan(data['longitude'][i]):
        newlonglist.append(round(np.nanmean(np.array(lst1[data['country'][i]])),2))
    else:
        newlonglist.append(data['longitude'][i])
data['longitude']=newlonglist
data.to_csv("EuCitiesTemperatures.csv", columns=data.columns)

In [6]:
def subset(data):
    cities=[]
    countries=list(data['country'].values)
    countries1=list()
    for i in data.index:
        if data['latitude'][i]>=40 and data['latitude'][i]<=60 and data['longitude'][i]>=15 and data['longitude'][i]<=30:
            cities.append(data['city'][i])
        else:
            countries1.append(countries[i])
            countries[i]=0
    lst = [i for i in countries if i != 0]
    maximalc=list()
    for i in lst:
        if i not in countries1:
            if i not in maximalc:
                maximalc.append(i)
    return cities, maximalc #list of cities that lie between latitudes 40 to 60 and longitudes 15 to 30, and a list countries have the maximal number of cities in this geographical band


In [8]:
def average_temp(data):
    eu=[] #eu only
    euc=[] #eu and coast
    c=[] #coast only
    n=[] #neither
    for i in data.index:
        if data['coastline'][i]=='yes' and data['EU'][i]=='no':
            c.append(data['temperature'][i])
        elif data['EU'][i]=='yes' and data['coastline'][i]=='no':
            eu.append(data['temperature'][i])
        elif data['coastline'][i]=='yes' and data['EU'][i]=='yes':
            euc.append(data['temperature'][i])
        elif data['coastline'][i]=='no' and data['EU'][i]=='no':
            n.append(data['temperature'][i])
    for i in data.index:
        if math.isnan(data['temperature'][i]):
            if data['coastline'][i]=='yes' and data['EU'][i]=='no':
                data['temperature'][i]=round(np.nanmean(np.array(c)),2)
            elif data['EU'][i]=='yes' and data['coastline'][i]=='no':
                data['temperature'][i]=round(np.nanmean(np.array(eu)),2)
            elif data['coastline'][i]=='yes' and data['EU'][i]=='yes':
                data['temperature'][i]=round(np.nanmean(np.array(euc)),2)
            elif data['coastline'][i]=='no' and data['EU'][i]=='no':
                data['temperature'][i]=round(np.nanmean(np.array(n)),2)
    data.to_csv("EuCitiesTemperatures.csv", columns=data.columns)
average_temp(data)

In [18]:
def showregions(data):
    regions=['Coast','EU','EU and coast','Neither']
    ceu=0 #counter eu only
    ceuc=0 #counter eu and coast
    cc=0 #counter coast only
    cn=0 #counter neither
    for i in data.index:
        if data['coastline'][i]=='yes' and data['EU'][i]=='no':
             cc+=1
        elif data['EU'][i]=='yes' and data['coastline'][i]=='no':
             ceu+=1
        elif data['coastline'][i]=='yes' and data['EU'][i]=='yes':
             ceuc+=1
        elif data['coastline'][i]=='no' and data['EU'][i]=='no':
            cn+=1
    cities=[cc,ceu,ceuc,cn]
    plt.bar(regions,cities)
    plt.show()

In [92]:
def eumap(data):
    countries=data.groupby('country')
    for name, group in countries:
        plt.plot(group.longitude, group.latitude, marker='o', linestyle='', markersize=5, label=name)

In [122]:
def pophist(data):
    countriespop=Series(data['population'].values, index=list(data['country']))
    countriespop = (countriespop.reset_index().drop_duplicates(subset='index', keep='last').set_index('index').sort_index())
    countriespop.hist(bins=5)

In [164]:
def subplots(data):
    fig, axes=plt.subplots(2,2,figsize=(30,30))
    ceu=0 #counter eu only
    ceuc=0 #counter eu and coast
    cc=0 #counter coast only
    cn=0 #counter neither
    for i in data.index:
        if data['coastline'][i]=='yes' and data['EU'][i]=='no':
            if data['temperature'][i]>10:
                color='red'
            elif data['temperature'][i]<6:
                color='blue'
            else:
                color='orange'
            axes[0][0].plot(data['city'][i],data['latitude'][i], 'o', c=color)
            cc+=1
        elif data['EU'][i]=='yes' and data['coastline'][i]=='no':
            if data['temperature'][i]>10:
                color='red'
            elif data['temperature'][i]<6:
                color='blue'
            else:
                color='orange'
            axes[0][1].plot(data['city'][i],data['latitude'][i], 'o', c=color)
            ceu+=1
        elif data['coastline'][i]=='yes' and data['EU'][i]=='yes':
            if data['temperature'][i]>10:
                color='red'
            elif data['temperature'][i]<6:
                color='blue'
            else:
                color='orange'
            axes[1][0].plot(data['city'][i],data['latitude'][i], 'o', c=color)
            ceuc+=1
        elif data['coastline'][i]=='no' and data['EU'][i]=='no':
            if data['temperature'][i]>10:
                color='red'
            elif data['temperature'][i]<6:
                color='blue'
            else:
                color='orange'
            axes[1][1].plot(data['city'][i],data['latitude'][i], 'o', c=color)
            cn+=1
    axes[0][0].set_title('Coastline only')
    axes[0][1].set_title('EU only')
    axes[1][0].set_title('Coastline and EU')
    axes[1][1].set_title('Neither')
    axes[0][0].set_xticks(range(cc))
    axes[0][1].set_xticks(range(ceu))
    axes[1][0].set_xticks(range(ceuc))
    axes[1][1].set_xticks(range(cn))
    axes[0][0].set_xticklabels(range(cc))
    axes[0][1].set_xticklabels(range(ceu))
    axes[1][0].set_xticklabels(range(ceuc))
    axes[1][1].set_xticklabels(range(cn))
    plt.show()